# ActBlue donor profile

In [14]:
import json
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import os

In [15]:
%matplotlib inline

plt.style.use('ggplot')

pd.options.display.float_format = '{:,.2f}'.format

In [16]:
def read_or_save(name, func):
    path = 'pickles/' + name + '.pickle.gz'
    if (os.path.isfile(path)):
        return pd.read_pickle(path)
    else:
        result = func()
        os.makedirs('pickles', exist_ok=True)
        result.to_pickle(path)
        return result

In [17]:
committee_id = "C00401224"
year_since = 2013

### Connect to the PostgreSQL database

In [18]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf['dbname'],
    user=conf['user'],
    host=conf['host'],
    password=conf['password']
)
conn.autocommit = True

In [19]:
actblue_contribs_count = read_or_save(
    'actblue_contribs_count',
    lambda: pd.read_sql("""
    -- DROP MATERIALIZED VIEW actblue_contribs;

    CREATE MATERIALIZED VIEW IF NOT EXISTS actblue_contribs AS
    SELECT contribs.*,
           fec_expenditures.election_code,
           beneficiary_committee_fec_id,
           beneficiary_candidate_fec_id,
           beneficiary_candidate_state,
           beneficiary_candidate_office,
           coalesce(cands.cand_id,cand_comms.cand_id) AS cand_id,
           coalesce(cands.cand_pty_affiliation,cand_comms.cand_pty_affiliation) AS cand_pty_affiliation,
           coalesce(cands.cand_office_st,cand_comms.cand_office_st) AS cand_office_st,
           coalesce(cands.cand_office,cand_comms.cand_office) AS cand_office,
           coalesce(cands.cand_pcc,cand_comms.cand_pcc) AS cand_pcc
    FROM
      (SELECT fec_contributions.filing_id,
              transaction_id,
              contributor_last_name,
              contributor_first_name,
              contributor_street_1,
              contributor_city,
              contributor_state,
              contributor_zip_code,
              contribution_date,
              contribution_amount,
              contributor_employer,
              contributor_occupation,
              memo_text_description
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = 'C00401224'
              AND extract(YEAR
                          FROM coverage_through_date) >= 2013
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI') AS contribs
    LEFT JOIN fec_expenditures ON fec_expenditures.filing_id IN
      (SELECT filing_id
       FROM fec_pac_summaries
       JOIN fec_amended_filings USING (filing_id)
       WHERE filer_committee_id_number = 'C00401224'
         AND extract(YEAR
                     FROM coverage_through_date) >= 2013
       ORDER BY coverage_through_date DESC)
    AND fec_expenditures.form_type = 'SB23'
    AND 'SB23_' || replace(transaction_id,'SA11AI_','') = transaction_id_number
    LEFT JOIN
      (SELECT DISTINCT ON (cand_id) cand_id,
                          cand_pcc,
                          cand_pty_affiliation,
                          cand_office_st,
                          cand_office
       FROM fec_candidates
       ORDER BY cand_id,
                cand_election_yr::int DESC) AS cands ON cands.cand_id = beneficiary_candidate_fec_id
    LEFT JOIN
      (SELECT DISTINCT ON (cand_pcc) cand_id,
                          cand_pcc,
                          cand_pty_affiliation,
                          cand_office_st,
                          cand_office
       FROM fec_candidates
       ORDER BY cand_pcc,
                cand_election_yr::int DESC) AS cand_comms ON cand_comms.cand_pcc = beneficiary_committee_fec_id; -- CREATE INDEX ON actblue_contribs ()

    GRANT ALL ON TABLE actblue_contribs TO redash_default;
    GRANT ALL ON TABLE actblue_contribs TO politics;

    ANALYZE actblue_contribs;

    SELECT count(*)
    FROM actblue_contribs;
    """, con=conn)
)
actblue_contribs_count

,count
0,73813481


### Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?

In [20]:
actblue_states = read_or_save(
    'actblue_states',
    lambda: pd.read_sql("""
    SELECT contributor_state,
           count(*),
           sum(contribution_amount)
    FROM actblue_contribs
    GROUP BY contributor_state
    """, con=conn)
)
actblue_states.sort_values(by=['sum'], ascending=False)

,contributor_state,count,sum
17,CA,13929247,"335,612,874.58"
64,NY,6454081,"186,707,526.03"
40,MA,3187355,"87,279,278.03"
88,TX,3732662,"81,421,346.63"
24,FL,3660635,"76,552,531.25"
34,IL,2820449,"64,434,206.45"
95,WA,3152336,"59,701,863.43"
91,VA,1903070,"53,151,404.77"
70,PA,2476800,"51,711,270.50"
42,MD,1692652,"46,873,506.68"


In [21]:
actblue_states.to_csv('data/states.csv')

In [22]:
clinton_votes = pd.read_csv('data/clintonvotes.csv')

clinton_votes

votes_vs_contribs = actblue_states.merge(clinton_votes, left_on='contributor_state', right_on='postal', how='inner')

votes_vs_contribs['dollars_per'] = votes_vs_contribs['sum']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['dollars_per'], ascending=False)

,contributor_state,count,sum,state,postal,votes,dollars_per
7,DC,549492,"35,476,385.41",District of Columbia,DC,282830,125.43
46,VT,551998,"10,521,946.09",Vermont,VT,178573,58.92
26,MT,363475,"8,862,784.24",Montana,MT,177709,49.87
19,MA,3187355,"87,279,278.03",Massachusetts,MA,1995196,43.74
32,NM,902815,"16,747,585.64",New Mexico,NM,385234,43.47
34,NY,6454081,"186,707,526.03",New York,NY,4556142,40.98
0,AK,227983,"4,492,464.77",Alaska,AK,116454,38.58
4,CA,13929247,"335,612,874.58",California,CA,8753792,38.34
30,NH,548011,"12,749,122.15",New Hampshire,NH,348526,36.58
50,WY,86714,"2,033,999.29",Wyoming,WY,55973,36.34


In [23]:
votes_vs_contribs['contribs_per'] = votes_vs_contribs['count']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['contribs_per'], ascending=False)

,contributor_state,count,sum,state,postal,votes,dollars_per,contribs_per
46,VT,551998,"10,521,946.09",Vermont,VT,178573,58.92,3.09
32,NM,902815,"16,747,585.64",New Mexico,NM,385234,43.47,2.34
37,OR,2187875,"33,864,200.42",Oregon,OR,1002106,33.79,2.18
26,MT,363475,"8,862,784.24",Montana,MT,177709,49.87,2.05
0,AK,227983,"4,492,464.77",Alaska,AK,116454,38.58,1.96
7,DC,549492,"35,476,385.41",District of Columbia,DC,282830,125.43,1.94
47,WA,3152336,"59,701,863.43",Washington,WA,1742718,34.26,1.81
19,MA,3187355,"87,279,278.03",Massachusetts,MA,1995196,43.74,1.60
4,CA,13929247,"335,612,874.58",California,CA,8753792,38.34,1.59
30,NH,548011,"12,749,122.15",New Hampshire,NH,348526,36.58,1.57


In [24]:
actblue_in_state = read_or_save(
    'actblue_in_state',
    lambda: pd.read_sql("""
    SELECT CASE
               WHEN coalesce(beneficiary_candidate_state,cand_office_st) = contributor_state THEN TRUE
               ELSE FALSE
           END AS in_state,
           sum(contribution_amount),
           count(*)
    FROM actblue_contribs
    WHERE beneficiary_candidate_state IS NOT NULL
      OR cand_office_st IS NOT NULL
    GROUP BY in_state
    """, con=conn)
)
actblue_in_state

,in_state,sum,count
0,False,"620,773,641.96",23794614
1,True,"330,828,769.69",5974546


In [25]:
actblue_freq = read_or_save(
    'actblue_freq',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count
    FROM
      (SELECT contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5),
              count(*) AS COUNT
       FROM actblue_contribs
       GROUP BY contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    """, con=conn)
)
actblue_freq

,count,count_of_count
0,1798,1
1,1489,3
2,1269,1
3,652,21
4,273,160
5,51,5705
6,2574,1
7,951,3
8,1898,2
9,70,3081


In [26]:
actblue_addicts = read_or_save(
    'actblue_addicts',
    lambda: pd.read_sql("""
    SELECT contributor_first_name,
           contributor_last_name,
           array_agg(DISTINCT contributor_occupation),
           array_agg(DISTINCT contributor_employer),
           array_agg(DISTINCT contributor_street_1),
           array_agg(DISTINCT contributor_city),
           contributor_state,
           left(contributor_zip_code,5) AS contributor_zip_code,
           count(*) AS COUNT,
           sum(contribution_amount) AS total,
           sum(contribution_amount)/count(*) AS avg_per
    FROM actblue_contribs
    WHERE extract(YEAR
                  FROM contribution_date) > 2016
    GROUP BY contributor_first_name,
             contributor_last_name,
             contributor_state,
             left(contributor_zip_code,5)
    ORDER BY COUNT DESC
    LIMIT 100
    """, con=conn)
)
actblue_addicts



,contributor_first_name,contributor_last_name,array_agg,array_agg,array_agg,array_agg,contributor_state,contributor_zip_code,count,total,avg_per
0,SIBYLLE,BARLOW,[RETIRED],[RETIRED],[241 HOLDEN WOOD ROAD],[CONCORD],MA,01742,14716,"54,401.98",3.70
1,TERRY,WOLFE,[RETAIL],"[LOWES, LOWE'S]","[1199 E 3RD ST, 1199 EAST THIRD STREET]",[MORGANTOWN],WV,26508,9456,"26,794.07",2.83
2,JOHN,COMELLA,[NONE],[NONE],[1900 J F KENNEDY BLVD SUITE 1624],[PHILADELPHIA],PA,19103,9359,"26,981.09",2.88
3,RICHARD,GOLDSTEIN,"[NOT EMPLOYED, RETIRED]","[NOT EMPLOYED, RETIRED]","[375 SAGAPONACK RD, L O BOX752, P O 752, POB0X...","[SAFAPONACK, SAGAPONACK]",NY,11962,8402,"146,293.73",17.41
4,VICKI,FARRAR,[NOT EMPLOYED],[NOT EMPLOYED],[P O BOX 140375],[GARDEN CITY],ID,83714,7856,"34,069.16",4.34
5,ROXANNE,WARREN,"[ARCHITECT, ARCHITECTS]",[SELF],[523 WEST 112TH STREET 72],[NEW YORK],NY,10025,7776,"79,123.94",10.18
6,CAROLINE,MERRIAM,[FOUNDATION PRESIDENT],[RAMSAY MERRIAM FUND],[1316 30TH STREET NW],[WASHINGTON],DC,20007,7710,"112,502.66",14.59
7,WILLIAM,CHEEK,[RETIRED PROFESSOR],[NONE],[6209 ESTELLE ST],[SAN DIEGO],CA,92115,7270,"85,805.11",11.80
8,MARTHA,UTZ,[NOT EMPLOYED],[NOT EMPLOYED],[1118 ALLSTON CT],[SAN JOSE],CA,95120,6996,"24,811.81",3.55
9,LUCY,HARMON,[NOT EMPLOYED],[SELF],[13621SHANNON ST],[LINDALE],TX,75771,6712,"59,341.99",8.84


### Where do contributors give from?

### Spreadsheet of the 1,307 candidate committees that got at least some money through ActBlue this cycle.

### Questions to answer
> - Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?
> - Where is the money going? Are people giving to local candidates or are they giving to candidates around the country? Which campaigns/candidates/groups have done the best job tapping into this network?
> - What’s the gender breakdown and how does it compare to the gender breakdown of itemized contributions? Is it different?
> - Are these new donors?
> - Can we tell if these are recurring contributions or one time contributions?
> - Do the majority of these donors give repeatedly or are they one and done? How does that compare to itemized contributions? How “sticky” is this?
